# Branch & Bound per Knapsack

In [65]:
import numpy as np

## Istanza di Knapsack

In [66]:
N = 5
P = np.array([5, 8, 8, 3, 1])
W = np.array([3, 7, 9, 4, 2])
C = 12

#N = 10
#P = np.array([10, 5, 4, 5, 3, 6, 2, 2, 4, 1])
#W = np.array([10, 5, 4, 6, 4, 8, 3, 4, 10, 5])
#C = 40

## Implementazione State Space Tree

In [67]:
class Node:
    def __init__(self, label):
        self.label = label
        self.children = []
        self.parent = None

    def setParent(self, parent):
        self.parent = parent

    def addChildren(self, children):
        self.children.extend(children)

    def __str__(self):
        return str(self.label)
        
class StateSpaceTree:
    def __init__(self, max_length, values):
        self.root = Node([])
        self.nodes = [self.root]
        self.max_length = max_length
        self.values = values

    def expand(self, node):
        if len(node.label) == self.max_length:
            return []
        children = []
        for value in self.values:
            label = node.label + [value]
            child = Node(label)
            child.setParent = node
            children.append(child)
        node.addChildren(children)
        self.nodes.extend(children)
        return children

## Implementazione Lista dei Live Node

### Definizione funzioni guadagno

In [68]:
def fw(node):
    x = np.array(node.label)
    return np.sum(x*W[0:len(x)])

def fp(node):
    x = np.array(node.label)
    return np.sum(x*P[0:len(x)])

def g(node):
    x = np.array(node.label)
    i = len(x)
    res = 0
    tot_weight = np.sum(x*W[0:i])
    while i < N:
        if tot_weight + W[i] <= C:
            res += P[i]
            tot_weight += W[i]
            i += 1
        else:
            res += (C - tot_weight) / W[i] * P[i]
            break
    return res

### Definizione classe Lista

In [69]:
class LiveNodesList:
    def __init__(self, root, strategy):
        self.live_nodes = [root]
        self.strategy = strategy

    def select(self):
        if self.strategy == "LIFO":
            last = self.live_nodes[-1]
            self.live_nodes = self.live_nodes[:-1]
            return last
        if self.strategy == "FIFO":
            first = self.live_nodes[0]
            self.live_nodes = self.live_nodes[1:]
            return first
        if self.strategy == "LC":
            max_node = self.live_nodes[0]
            max_profit = fp(max_node) + g(max_node)
            for node in self.live_nodes[1:]:
                profit = fp(node) + g(node)
                if profit > max_profit:
                    max_profit = profit
                    max_node = node
            self.live_nodes.remove(max_node)
            return max_node
                
    def insert(self, node):
        self.live_nodes.append(node)

    def empty(self):
        return len(self.live_nodes) == 0

    def __str__(self):
        return str([node.label for node in self.live_nodes])

## Algoritmo di visita

In [70]:
def branchAndBoundForKP(tree):
    z_max, x_max = initializeMaxSolution()
    live_nodes = LiveNodesList(tree.root, strategy="LC")
    iteration = 0
    while not live_nodes.empty():
        print(f"\nIteration {iteration}")
        print(f"Live nodes: {live_nodes}")
        e_node = live_nodes.select()
        print(f"E-node: {e_node}")
        z = fp(e_node)
        if not (fw(e_node) > C or (z + g(e_node)) <= z_max):
            if z >= z_max:
                z_max = z
                x_max = node.label
                print(f"A new max has been found: {z_max}")
            print("The E-node will be expanded")
            new_nodes = tree.expand(e_node)
            for node in new_nodes:
                live_nodes.insert(node)
        else:
            print("The E-node will not be expanded.")
        iteration += 1
    return x_max, z_max

def initializeMaxSolution():
    z = 0
    x = [0]*N
    tot_weight = 0
    i = 0
    while tot_weight + W[i] <= C:
        z += P[i]
        tot_weight += W[i]
        x[i] = 1
        i += 1
    return z, x

tree = StateSpaceTree(5, [0,1])
z_star = branchAndBoundForKP(tree)
print(f"\nThe answer is: {z_star[0]} with profit {z_star[1]} and total weight: {np.sum(z_star[0]*W)}")


Iteration 0
Live nodes: [[]]
E-node: []
The E-node will not be expanded.

The answer is: [1, 1, 1, 1, 1, 1, 1, 0, 0, 0] with profit 35 and total weight: 40
